## Predict Amazon Product Prices

### An example fine-tuned model using Llama3.1

### Resuming fine-tuning from a checkpoint saved to the HuggingFace Hub



In [1]:
# pip installs

!pip install -q datasets requests torch peft bitsandbytes transformers trl accelerate sentencepiece wandb matplotlib

In [2]:
# imports

import os
import re
import math
from tqdm import tqdm
from google.colab import userdata
from huggingface_hub import login
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, set_seed
from datasets import load_dataset, Dataset, DatasetDict
import wandb
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
# Constants

BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"
PROJECT_NAME = "pricer"
HF_USER = "zoya-hammadk"
DATASET_NAME = f"{HF_USER}/pricer-data"
ORIGINAL_FINETUNED = "zoya-hammadk/pricer-2025-04-13_04.15.55"
ORIGINAL_REVISION = "ed00b02a93780f710094f8a45219d9e5db0945c1" # the commit hash in the HuggingFace repo
RUN_NAME =  f"{datetime.now():%Y-%m-%d_%H.%M.%S}"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
HUB_MODEL_NAME = f"{HF_USER}/{PROJECT_RUN_NAME}"


# Hyperparameters for QLoRA Fine-Tuning

EPOCHS = 1
LORA_ALPHA = 64
LORA_R = 32
LORA_DROPOUT = 0.1
BATCH_SIZE = 16
GRADIENT_ACCUMULATION_STEPS = 1
LEARNING_RATE = 2e-5 # 1e-4
LR_SCHEDULER_TYPE = 'cosine'
WEIGHT_DECAY = 0.001
TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]
MAX_SEQUENCE_LENGTH = 182
QUANT_4_BIT = True

# Other config

STEPS = 50
SAVE_STEPS = 2000
LOG_TO_WANDB = True

# Used for writing to output in color

GREEN = "\033[92m"
YELLOW = "\033[93m"
RED = "\033[91m"
RESET = "\033[0m"

%matplotlib inline

In [4]:
# Log in to HuggingFace

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [5]:
# Log in to Weights & Biases
wandb_api_key = userdata.get('WANDB_API_KEY')
os.environ["WANDB_API_KEY"] = wandb_api_key
wandb.login()

# Configure Weights & Biases to record against our project
os.environ["WANDB_PROJECT"] = PROJECT_NAME
os.environ["WANDB_LOG_MODEL"] = "checkpoint" if LOG_TO_WANDB else "end"
os.environ["WANDB_WATCH"] = "gradients"

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: zoya5a-bclr (zoya5a-bclr-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [6]:
dataset = load_dataset(DATASET_NAME)
train = dataset['train']
test = dataset['test']

In [8]:
if LOG_TO_WANDB:
  details = {
      "base_model": BASE_MODEL,
      "dataset": DATASET_NAME,
      "size": len(train),
      "epochs": EPOCHS,
      "alpha": LORA_ALPHA,
      "r": LORA_R,
      "dropout": LORA_DROPOUT,
      "batch_size": BATCH_SIZE,
      "gradient_accumulation": GRADIENT_ACCUMULATION_STEPS,
      "lr": LEARNING_RATE,
      "lr_scheduler": LR_SCHEDULER_TYPE,
      "max_sequence_length": MAX_SEQUENCE_LENGTH,
      "quant_4_bit": QUANT_4_BIT
  }

  wandb.init(
      project=PROJECT_NAME,
      name=RUN_NAME,
      config=details
  )

In [9]:
# pick the right quantization

if QUANT_4_BIT:
  quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
  )
else:
  quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.bfloat16
  )

In [ ]:
# Load the Tokenizer and the Model

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
    use_cache=False
)
base_model.generation_config.pad_token_id = tokenizer.pad_token_id

print(f"Memory footprint: {base_model.get_memory_footprint() / 1e6:.1f} MB")

# Load the Tokenizer and the Model

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)
base_model.generation_config.pad_token_id = tokenizer.pad_token_id

# Load the fine-tuned model with PEFT

fine_tuned_model = PeftModel.from_pretrained(base_model, ORIGINAL_FINETUNED, revision=ORIGINAL_REVISION, is_trainable=True)
fine_tuned_model.train()

print(f"Memory footprint: {fine_tuned_model.get_memory_footprint() / 1e6:.1f} MB")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Memory footprint: 5591.5 MB


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Memory footprint: 5700.6 MB


#### Resuming Fine-Tuning



In [11]:
from trl import DataCollatorForCompletionOnlyLM
response_template = "Price is $"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [ ]:
# First, specify the configuration parameters for LoRA

peft_parameters = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    r=LORA_R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=TARGET_MODULES,
)

# Next, specify the general configuration parameters for training

train_params = SFTConfig(
    output_dir=PROJECT_RUN_NAME,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=1,
    eval_strategy="no",
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    optim="paged_adamw_32bit",
    save_steps=SAVE_STEPS,
    save_total_limit=10,
    logging_steps=STEPS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    report_to="wandb" if LOG_TO_WANDB else None,
    run_name=RUN_NAME,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    dataset_text_field="text",
    save_strategy="steps",
    hub_strategy="every_save",
    push_to_hub=True,
    hub_model_id=HUB_MODEL_NAME,
    hub_private_repo=True,
)

# And now, the Supervised Fine Tuning Trainer will carry out the fine-tuning
# Given these 2 sets of configuration parameters

fine_tuning = SFTTrainer(
    model=fine_tuned_model,
    train_dataset=train,
    peft_config=peft_parameters,
    args=train_params,
    data_collator=collator,
)

# Fine-tune!
fine_tuning.train()

# Push our fine-tuned model to Hugging Face
fine_tuning.model.push_to_hub(PROJECT_RUN_NAME, private=True)
print(f"Saved to the hub: {PROJECT_RUN_NAME}")

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
50,1.462800
100,1.437600
150,1.396200
200,1.495900
250,1.439000
300,1.422900
350,1.456500
400,1.437800
450,1.435300
500,1.454300


wandb: Adding directory to artifact (./pricer-2025-04-15_01.54.18/checkpoint-2000)... Done. 0.8s
wandb: Adding directory to artifact (./pricer-2025-04-15_01.54.18/checkpoint-4000)... Done. 0.7s
wandb: Adding directory to artifact (./pricer-2025-04-15_01.54.18/checkpoint-6000)... Done. 0.7s
wandb: Adding directory to artifact (./pricer-2025-04-15_01.54.18/checkpoint-6250)... Done. 0.8s
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


README.md:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Saved to the hub: pricer-2025-04-15_01.54.18


In [13]:
if LOG_TO_WANDB:
  wandb.finish()

train/epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train/grad_norm,▂▆▁▂▁▂▂▄▃▁▂▄▃▄▂▄▄▄▄▂▄▄▆▄▅▅▃▃▆▅▆██▃▄▃▇▄▄▆
train/learning_rate,█████▇▇▇▇▇▆▆▆▆▆▅▅▅▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁
train/loss,█▆▆▆▆▃▆▆▆▇▆▇▅▄▆▄▅▄▂▆▇▃▅▄▄▃▂▄▅▄▆▁▃▇▂▂▅▁▅▇
train/mean_token_accuracy,▅▄▂▂▂▂▅▃▂▁▅▄▂▅▅▄▃▂▃▆▅▂▃▄▄▄▃▄▄▄▁▆█▅▃▄▄▆▄▅
train/num_tokens,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇███
total_flos,8.07816640442794e+17
train/epoch,1
train/global_step,6250
train/grad_norm,3.30021
